In [12]:
from Bio.PDB import *
from utils.carb_utils import *
#import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix as dm


#-alternate_3_letter_codes pdb_sugar
#-out:level 100

#-beta
#-auto_detect_glycan_connections
#-alternate_3_letter_codes pdb_sugar



import os
import numpy as np
import pandas as pd
import copy

BOND_CUT = 1.85
INTERACT = 4.5

input_dir = "./"
os.chdir(input_dir)

In [4]:
ls = os.listdir('./pdbs')
print(len(ls))

6252


In [5]:
gly = '''N-acetyl-beta-D-glucosamine (NAG), beta-D-glucopyranose (BGC),
alpha-D-glucopyranose (GLC), alpha-D-mannopyranose (MAN),
beta-D-galactopyranose (GAL), alpha-L-fucopyranose (FUC),
beta-D-mannopyranose (BMA), N-acetyl-alpha-D-mannosamine (BM3), nonyl
beta-D-glucopyranoside (BNG), beta-D-xylopyranose (XYP),
uridine-diphosphate-n-acetylglucosamine (UD1), N-acetyl-alpha-D-galactosamine
(A2G), undecyl-maltoside (UMQ), N-acetyl-beta-D-galactosamine (NGA),
galactose-uridine-5-diphosphate (GDU), 6-O-phosphono-beta-D-glucopyranose
(BG6), N-acetyl-beta-neuraminic acid (SLB), N-acetyl-alpha-neuraminic acid (SIA), (NEU),
fructose -6-phosphate (F6R), alpha-D-galactopyranose (X6X),
1,6-di-O-phosphono-D-fructose (P6F), 2-amino-2-deoxy-alpha-D-glucopyranose
(PA1), 2-amino-2-deoxy-beta-D-galactopyranose (1GN), alpha-L-rhamnopyranose
(RAM), alpha-D-Abequopyranose (ABE), alpha-D-glucopyranuronic acid (GCU),
2-acetamido-2-deoxy-4-O-sulfo-beta-D-galactopyranose (ASG),
alpha-L-gulopyranuronic acid (LGU), alpha-L-arabinofuranose (AHR),
beta-D-galactofuranuronic acid (GTK), beta-D-glucopyranuronic acid (BDP),
beta-L-fructofuranose (LFR) and cyclodextrins, beta-D-ribopyranose (RIP), '''

carb = [];
gly = gly.split('(')
#print(gly)
for jj in range(len(gly)):
    if ')' in gly[jj]:
        carb.append(gly[jj].split(')')[0])
print(carb)

['NAG', 'BGC', 'GLC', 'MAN', 'GAL', 'FUC', 'BMA', 'BM3', 'BNG', 'XYP', 'UD1', 'A2G', 'UMQ', 'NGA', 'GDU', 'BG6', 'SLB', 'SIA', 'NEU', 'F6R', 'X6X', 'P6F', 'PA1', '1GN', 'RAM', 'ABE', 'GCU', 'ASG', 'LGU', 'AHR', 'GTK', 'BDP', 'LFR', 'RIP']


In [6]:
aa_inv = {
    "H": "HIS",
    "K": "LYS",
    "R": "ARG",
    "D": "ASP",
    "E": "GLU",
    "S": "SER",
    "T": "THR",
    "N": "ASN",
    "Q": "GLN",
    "A": "ALA",
    "V": "VAL",
    "L": "LEU",
    "I": "ILE",
    "M": "MET",
    "F": "PHE",
    "Y": "TYR",
    "W": "TRP",
    "P": "PRO",
    "G": "GLY",
    "C": "CYS",
    "X": "MSE",
    "Z": "CYD"
}
aa_inv.keys()
prot_aa = aa_inv.values()

In [7]:
def get_coor(structure,carb,prot_aa):
    coor_c = []
    coor_p = []
    res_c = []
    res_p = []

    coor_x = []
    res_x = []

    models = structure.get_models()
    models = list(models)
    for m in range(len(models)):
        chains = list(models[m].get_chains())
        for c in range(len(chains)):
            residues = list(chains[c].get_residues())
            for r in range(len(residues)):
                res = residues[r].get_resname()
                if res == 'HOH':
                    continue;

                atoms = list(residues[r].get_atoms())

                for a in range(len(atoms)):
                    at = atoms[a]

                    if 'H' in at.get_name():
                        continue;

                    #print(str(residues[r].get_parent().id).strip())

                    if res in carb:
                        coor_c.append( at.get_coord() )
                        res_c.append( [ str(residues[r].id[1]).strip(), str(chains[c].id).strip(), str(residues[r].get_resname()) ] )
                        #res_c.append( [ str(residues[r].id[1]).strip(), str(residues[r].get_resname()) ] )
                    elif res in prot_aa:
                        coor_p.append( at.get_coord() )
                        res_p.append( [ str(residues[r].id[1]).strip(), str(chains[c].id).strip(), str(residues[r].get_resname()) ] )
                        #res_p.append( [ str(residues[r].id[1]).strip(), str(residues[r].get_resname()) ] )
                    else:
                        coor_x.append( at.get_coord() )
                        res_x.append( [ str(residues[r].id[1]).strip(), str(chains[c].id).strip(), str(residues[r].get_resname()) ] )
                        #res_x.append( [ str(residues[r].id[1]).strip(), str(residues[r].get_resname()) ] )
                        
        return np.array(coor_c), np.array(coor_p), np.array(coor_x), res_c, res_p, res_x

In [8]:
def remove_glycosylated(coor_c,coor_p,res_c,res_p,BOND_CUT=1.85):

    d = dm(coor_c,coor_p) < BOND_CUT
    a = np.array( np.where(d == 1) )
    a = np.array(a)

    #a = np.array( np.where(d == 1) )
    bad_c = []
    for ii in range(a.shape[1]):
        bad_c.append( res_c[a[0,ii]] )
        #print(res_c[a[0,ii]], res_p[a[1,ii]])


    pre_bad = bad_c;

    while True:      
        ccd = dm(coor_c,coor_c) < BOND_CUT
        ccd = np.array( np.where(ccd == 1) )
        ccd = np.array(ccd)
        #print(ccd)

        for ii in range(ccd.shape[1]):
            if res_c[ ccd[0,ii] ] in bad_c or res_c[ ccd[1,ii] ] in bad_c:
                if res_c[ccd[0,ii]] not in bad_c:
                    bad_c.append( res_c[ccd[0,ii]] )
                if res_c[ccd[1,ii]] not in bad_c:
                    bad_c.append( res_c[ccd[1,ii]] )
            #print(res_c[ ccd[0,ii] ], res_c[ccd[1,ii]])

        if pre_bad == bad_c:
            break;
        pre_bad = copy.deepcopy(bad_c)

    good_c = []
    for c in res_c:
        if c in bad_c:
            continue;
        if c not in good_c:
            good_c.append(c)

    return good_c


def add_nonglycans(coor_c,coor_x,good_c,res_c,res_x):
    #Add non-glycan connections
    weirdres = dm(coor_c,coor_x) < BOND_CUT
    weirdres = np.array( np.where(weirdres == 1) )
    weirdres =  np.array(weirdres)
    #print(weirdres)

    for ii in range(weirdres.shape[1]):
        if res_c[weirdres[0,ii]] in good_c:
            good_c.append( res_x[weirdres[1,ii]] )
    return good_c


In [9]:
res_p = []
good_c = []

class GlycanSelect(Select):
    def accept_residue(self, residue):
        r = residue
        #help(r)
        #if residue.get_resname()=='GLY':
        
        n = str(r.id[1]).strip()
        c = str(r.get_parent().id).strip()
        resname = str(r.get_resname())
        resarr = [n,c,resname]
        #if resname in carb:
        if resarr in good_c:
            return 1
        else:
            return 0
        

class ProtSelect(Select):
    def accept_residue(self, residue):
        r = residue
        #help(r)
        #if residue.get_resname()=='GLY':
        
        n = str(r.id[1]).strip()
        c = str(r.get_parent().id).strip()
        resname = str(r.get_resname())
        resarr = [n,c,resname]
        #if resname in carb:
        if resarr in res_p:
            return 1
        else:
            return 0
        
#io=PDBIO()
#io.set_structure(structure)
#io.save('prot_only.pdb', ProtSelect())



In [10]:
def find_interactChains(coor_c,coor_p,res_c,res_p,good_c,INTERACT=4.2):
    #determine chain-chain interactions
    d = dm(coor_c,coor_p) < INTERACT
    a = np.array( np.where(d == 1) )
    a = np.array(a)

    #chain_int = {}
    chain_int = ''
    for ii in range(a.shape[1]):
        if res_c[a[0,ii]] not in good_c:
            continue
        res1 = res_c[ a[0,ii] ]
        res2 = res_p[ a[1,ii] ]

        #print(res1,res2)

        #chain_int[res1[1]] = res2[1]
        ope = res1[1] + ',' + res2[1]
        if ope not in chain_int:
            chain_int +=ope + '\n'

        #chain_int.update({res1[1]:res2[2]})

        #bad_c.append( res_c[a[0,ii]] )
        #print(res_c[a[0,ii]], res_p[a[1,ii]])

    #chain_int

    return chain_int

In [11]:
bad = []

In [13]:
parser=PDBParser()
io=PDBIO()


for ii in range(1000):
    pdb = ls[ii]
    p = pdb.split('.')[0]
    
    #if ii == 2:
    #    break;
    try:
        structure=parser.get_structure("prot", 'pdbs/'+ pdb)
        print(ii,pdb)
    
    
    
        coor_c, coor_p, coor_x, res_c, res_p, res_x = get_coor(structure,carb,prot_aa)
        #print(len(res_c))
        good_c = remove_glycosylated(coor_c,coor_p,res_c,res_p,BOND_CUT=BOND_CUT)

        #print(len(good_c))
        if len(coor_x) > 0:
            good_c = add_nonglycans(coor_c,coor_x,good_c,res_c,res_x)
        #print(len(good_c))

        chain_int = find_interactChains(coor_c,coor_p,res_c,res_p,good_c,INTERACT=INTERACT)

        #output the pre-processed files
        io.set_structure(structure)
        io.save('./pdb_pre/' + p + '_prot.pdb', ProtSelect())
        io.save('./pdb_pre/' + p + '_carb.pdb', GlycanSelect())
        f = open('./pdb_pre/' + p + '_int.txt','w+')
        f.write(chain_int)
        f.close()
    except:
        bad.append(pdb)
    

/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 16199.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 16283.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 16311.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 16395.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain E is discontinuous at line 16423.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARN

0 1HGG.pdb
1 1drk.pdb
3 1O7Q.pdb
4 2ARB.pdb


/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 15115.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 15241.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 15379.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 16315.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4293.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNI

5 5AQ1.pdb
6 4BSE.pdb


/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2636.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3473.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5899.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 5904.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 5909.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: 

...

/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10372.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 10417.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 10459.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 10505.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3610.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNI

993 7VS7.pdb
994 3AMN.pdb
995 3H3K.pdb
996 6UQC.pdb


/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 4604.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:127: PDBConstructionWarning: WARNING: Residue (' ', 356, ' ') redefined at line 1736.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:127: PDBConstructionWarning: WARNING: Residue (' ', 364, ' ') redefined at line 1809.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:127: PDBConstructionWarning: WARNING: Residue (' ', 368, ' ') redefined at line 1845.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:127: PDBConstructionWarning: WARNING: Residue (' ', 370, ' ') redefined at line 1867.
  warnings.warn(
/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.

999 1GOR.pdb


/Users/scanner1/anaconda3/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 2713.
  warnings.warn(


In [30]:
parser=PDBParser()

#structure=parser.get_structure("prot", 'pdbs/'+ ls[0])
structure=parser.get_structure("prot", 'pdbs/'+ '1W3L.pdb')

/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 5919.
  warnings.warn(


In [60]:
parser=PDBParser()
io=PDBIO()

new_res = []
for ii in range(len(bad)):
    pdb = bad[ii]
    p = pdb.split('.')[0]
    
    #if ii == 2:
    #    break;
    
    structure=parser.get_structure("prot", 'pdbs/'+ pdb)
    #print(ii,pdb)



    coor_c, coor_p, coor_x, res_c, res_p, res_x = get_coor(structure,carb,prot_aa)
    #print(res_x)
    for jj in res_x:
        if jj[-1] not in new_res:
            new_res.append(jj[-1])
    #print(len(res_c))
    #good_c = remove_glycosylated(coor_c,coor_p,res_c,res_p,BOND_CUT=BOND_CUT)

    #print(len(good_c))
    #if len(coor_x) > 0:
    #    good_c = add_nonglycans(coor_c,coor_x,good_c,res_c,res_x)
    #print(len(good_c))

    #chain_int = find_interactChains(coor_c,coor_p,res_c,res_p,good_c,INTERACT=INTERACT)

    #output the pre-processed files
    #io.set_structure(structure)
    #io.save('./pdb_pre/' + p + '_prot.pdb', ProtSelect())
    #io.save('./pdb_pre/' + p + '_carb.pdb', GlycanSelect())
    #f = open('./pdb_pre/' + p + '_int.txt','w+')
    #f.write(chain_int)
    #f.close()
print(new_res)
    

/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10865.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 10886.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain C is discontinuous at line 10907.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain D is discontinuous at line 10928.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 10949.
  warnings.warn(
/Users/scanner1/opt/anaconda3/lib/python3.8/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstru

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte

In [ ]:
print(new_res)